# 🗑️ SISTEM PEMILAH SAMPAH CERDAS - YOLOV8M TRAINING

**Notebook Training untuk Google Colab Pro**

---

## 📋 Informasi Project

- **Model**: YOLOv8m (Medium)
- **Dataset**: Sampah Organik & Anorganik (dari Roboflow)
- **Target Kelas**: 3 (Organik, Anorganik, B3)
- **Hardware**: ESP32 + ESP32-CAM + Servo SG90
- **Tujuan**: Tugas UAS

---

## ⚡ Quick Start

1. Pastikan Runtime Type = **GPU** (Runtime > Change runtime type)
2. Ganti `YOUR_API_KEY_HERE` dengan Roboflow API key Anda
3. Run semua cells (Runtime > Run all)
4. Download `best.pt` dari cell terakhir

---

In [ ]:
# 🚀 SISTEM PEMILAH SAMPAH CERDAS - YOLOV8M TRAINING
# ================================================
# Notebook ini untuk training model YOLOv8m di Google Colab Pro
# Target: 3 Kelas (Organik, Anorganik, B3)
# Hardware: ESP32 + ESP32-CAM + Servo + Sensor Ultrasonik

print("=" * 60)
print("🗑️ SISTEM PEMILAH SAMPAH CERDAS - TRAINING")
print("=" * 60)

# Check GPU
import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n🚀 GPU siap digunakan untuk training!")
else:
    print("\n⚠️ WARNING: GPU tidak terdeteksi!")
    print("   Pastikan Runtime Type = GPU di menu Runtime > Change runtime type")

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install ultralytics roboflow -q
print("✅ Dependencies installed!")

import os
from pathlib import Path
from IPython.display import Image, display
from ultralytics import YOLO
import yaml

In [ ]:
# 💾 MOUNT GOOGLE DRIVE
# Untuk menyimpan hasil training secara permanen

from google.colab import drive
drive.mount('/content/drive')

# Buat folder project di Google Drive
PROJECT_DIR = Path('/content/drive/MyDrive/sistem-pemilah-sampah')
PROJECT_DIR.mkdir(exist_ok=True)

print(f"✅ Google Drive mounted!")
print(f"📁 Project directory: {PROJECT_DIR}")

# Pindah ke direktori kerja
os.chdir('/content')
print(f"📂 Working directory: {os.getcwd()}")

In [ ]:
# 📊 DOWNLOAD DATASET DARI ROBOFLOW
# Dataset: Sampah Organik, Anorganik, dan B3

from roboflow import Roboflow

print("=" * 60)
print("📥 DOWNLOADING DATASET")
print("=" * 60)

# ⚠️ GANTI DENGAN API KEY ANDA!
# Cara mendapatkan API key:
# 1. Daftar di https://roboflow.com (GRATIS)
# 2. Login ke https://app.roboflow.com
# 3. Klik Settings (kiri bawah) > Roboflow API
# 4. Copy Private API Key
# 5. Paste di bawah ini

ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # 👈 GANTI INI!

if ROBOFLOW_API_KEY == "YOUR_API_KEY_HERE":
    print("\n⚠️ ERROR: API Key belum diganti!")
    print("   Silakan dapatkan API key dari:")
    print("   https://app.roboflow.com/settings/api")
    print("\n   Atau download dataset manual:")
    print("   1. Buka: https://universe.roboflow.com/siscer-project/sampah-organik-dan-anorganik")
    print("   2. Klik 'Download Dataset'")
    print("   3. Pilih format: YOLOv8")
    print("   4. Upload ZIP ke Colab")
else:
    try:
        # Initialize Roboflow
        rf = Roboflow(api_key=ROBOFLOW_API_KEY)
        
        # Download dataset Sampah Organik & Anorganik
        print("\n📥 Downloading dataset dari Roboflow...")
        project = rf.workspace("siscer-project").project("sampah-organik-dan-anorganik")
        dataset = project.version(3).download("yolov8")
        
        DATASET_PATH = dataset.location
        print(f"\n✅ Dataset berhasil didownload!")
        print(f"📁 Location: {DATASET_PATH}")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        print("\n💡 Solusi:")
        print("   1. Cek API key sudah benar")
        print("   2. Pastikan internet stabil")
        print("   3. Atau download manual (lihat instruksi di atas)")

In [ ]:
# 🔍 EKSPLORASI DATASET
# Melihat struktur dan isi dataset

import os
import matplotlib.pyplot as plt
from PIL import Image
import random

print("=" * 60)
print("📊 DATASET EXPLORATION")
print("=" * 60)

# Baca data.yaml
yaml_path = Path(DATASET_PATH) / 'data.yaml'
with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print(f"\n📄 Dataset Configuration:")
print(f"   Classes: {data_config['nc']}")
print(f"   Names: {data_config['names']}")

# Hitung jumlah gambar
train_images = list(Path(DATASET_PATH).glob('train/images/*.jpg')) + \
               list(Path(DATASET_PATH).glob('train/images/*.png'))
val_images = list(Path(DATASET_PATH).glob('valid/images/*.jpg')) + \
             list(Path(DATASET_PATH).glob('valid/images/*.png'))

print(f"\n📈 Dataset Statistics:")
print(f"   Training images: {len(train_images)}")
print(f"   Validation images: {len(val_images)}")
print(f"   Total: {len(train_images) + len(val_images)}")

# Tampilkan sample gambar
print("\n🖼️ Sample Images:")
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

sample_images = random.sample(train_images, min(6, len(train_images)))
for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path)
    axes[idx].imshow(img)
    axes[idx].set_title(img_path.name, fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Update data.yaml untuk 3 kelas (jika perlu)
# Catatan: Dataset Roboflow biasanya hanya 2 kelas (organik & anorganik)
# Anda perlu menambahkan kelas B3 secara manual jika diperlukan

print("\n⚠️ CATATAN PENTING:")
print("   Dataset Roboflow biasanya hanya memiliki 2 kelas:")
print("   - Organik")
print("   - Anorganik")
print("\n   Untuk kelas B3 (Bahan Berbahaya & Beracun):")
print("   - Tambahkan gambar B3 secara manual")
print("   - Atau lanjutkan training dengan 2 kelas dulu")
print("\n   Untuk UAS besok, 2 kelas sudah cukup!")
print("   B3 bisa ditambahkan nanti.")

In [ ]:
# ⚙️ KONFIGURASI TRAINING
# Setup parameter training untuk YOLOv8m

print("=" * 60)
print("⚙️ TRAINING CONFIGURATION")
print("=" * 60)

CONFIG = {
    # Model
    'model': 'yolov8m.pt',      # YOLOv8 Medium (balance accuracy & speed)
    
    # Training
    'epochs': 150,               # Jumlah epoch (bisa dikurangi jika waktu terbatas)
    'imgsz': 640,                # Ukuran gambar
    'batch': 32,                 # Batch size untuk Colab Pro (T4/A100)
    'device': 0,                 # GPU device
    
    # Optimization
    'optimizer': 'AdamW',        # Optimizer (AdamW lebih baik dari Adam)
    'lr0': 0.001,               # Initial learning rate
    'lrf': 0.01,                # Final learning rate (fraction of lr0)
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'patience': 50,              # Early stopping patience
    
    # Augmentation
    'hsv_h': 0.015,             # Hue augmentation
    'hsv_s': 0.7,               # Saturation augmentation
    'hsv_v': 0.4,               # Value augmentation
    'degrees': 10.0,            # Rotation degree
    'translate': 0.1,           # Translation
    'scale': 0.5,               # Scaling
    'shear': 0.0,               # Shear
    'perspective': 0.0,         # Perspective
    'flipud': 0.0,              # Flip up-down
    'fliplr': 0.5,              # Flip left-right
    'mosaic': 1.0,              # Mosaic augmentation
    'mixup': 0.15,              # Mixup augmentation
    
    # Project
    'project': 'pemilah-sampah',
    'name': 'yolov8m_organik_anorganik',
}

print("\n📋 Training Parameters:")
for key, value in CONFIG.items():
    print(f"   {key:15} : {value}")

print("\n💡 Tips:")
print("   - Epochs 150 = ~1-1.5 jam di Colab Pro")
print("   - Bisa dikurangi jadi 100 jika waktu terbatas")
print("   - Model terbaik otomatis tersimpan di 'best.pt'")

In [ ]:
# 🚀 START TRAINING
# Proses training dimulai!

print("\n" + "=" * 60)
print("🚀 MEMULAI TRAINING")
print("=" * 60)
print("\n⏰ Estimasi waktu: 1-2 jam (tergantung GPU & dataset size)")
print("💡 Tips: Biarkan jalan, bisa sambil istirahat!\n")

# Load model
model = YOLO(CONFIG['model'])

# Start training
results = model.train(
    data=str(yaml_path),
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    device=CONFIG['device'],
    
    # Optimization
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    patience=CONFIG['patience'],
    
    # Augmentation
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    perspective=CONFIG['perspective'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
    
    # Validation & Saving
    val=True,
    save=True,
    save_period=10,
    plots=True,
    verbose=True,
    
    # Project
    project=CONFIG['project'],
    name=CONFIG['name'],
    exist_ok=False,
)

print("\n" + "=" * 60)
print("✅ TRAINING SELESAI!")
print("=" * 60)

In [ ]:
# 📈 EVALUASI MODEL
# Melihat performa model pada validation set

print("\n" + "=" * 60)
print("📊 EVALUASI MODEL")
print("=" * 60)

# Evaluate
metrics = model.val()

# Display metrics
print(f"\n📈 HASIL EVALUASI:")
print(f"   mAP50      : {metrics.box.map50:.4f}")
print(f"   mAP50-95   : {metrics.box.map:.4f}")
print(f"   Precision  : {metrics.box.mp:.4f}")
print(f"   Recall     : {metrics.box.mr:.4f}")

# Per-class metrics
if hasattr(metrics.box, 'ap50'):
    print(f"\n📊 Metrics per Kelas:")
    class_names = data_config['names']
    for i, name in enumerate(class_names):
        if i < len(metrics.box.ap50):
            print(f"   {name:12} - mAP50: {metrics.box.ap50[i]:.4f}")

print("\n💡 Interpretasi:")
print("   mAP50 > 0.85 = BAGUS SEKALI! 🎉")
print("   mAP50 > 0.70 = CUKUP BAIK ✅")
print("   mAP50 < 0.70 = Perlu improvement 📈")

In [ ]:
# 📊 VISUALISASI HASIL TRAINING
# Melihat training plots, confusion matrix, dan prediksi

run_dir = Path(CONFIG['project']) / CONFIG['name']

print("=" * 60)
print("📊 VISUALISASI HASIL")
print("=" * 60)

# 1. Training Results
results_img = run_dir / 'results.png'
if results_img.exists():
    print("\n📈 1. Training Metrics (Loss, Precision, Recall, mAP):")
    display(Image(filename=str(results_img)))

# 2. Confusion Matrix
confusion_img = run_dir / 'confusion_matrix.png'
if confusion_img.exists():
    print("\n🎯 2. Confusion Matrix:")
    display(Image(filename=str(confusion_img)))

# 3. Sample Predictions
val_batch = run_dir / 'val_batch0_pred.jpg'
if val_batch.exists():
    print("\n🔍 3. Sample Predictions pada Validation Set:")
    display(Image(filename=str(val_batch)))

print("\n✅ Semua visualisasi ditampilkan!")

In [ ]:
# 🧪 TEST INFERENCE
# Mencoba model pada gambar random dari validation set

print("=" * 60)
print("🧪 TEST INFERENCE")
print("=" * 60)

# Load best model
best_model = YOLO(run_dir / 'weights' / 'best.pt')

# Test pada beberapa gambar random
test_images = random.sample(val_images, min(4, len(val_images)))

fig, axes = plt.subplots(2, 2, figsize=(15, 15))
axes = axes.ravel()

print("\n🔍 Hasil Deteksi:\n")

for idx, img_path in enumerate(test_images):
    # Predict
    results = best_model(str(img_path), conf=0.5)
    
    # Plot
    annotated = results[0].plot()
    axes[idx].imshow(annotated[..., ::-1])  # BGR to RGB
    axes[idx].set_title(f"Prediction: {img_path.name}", fontsize=12)
    axes[idx].axis('off')
    
    # Print detections
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = data_config['names'][cls]
            print(f"   📸 {img_path.name}")
            print(f"      ➜ {class_name.upper()} ({conf:.2%})\n")
    else:
        print(f"   📸 {img_path.name}")
        print(f"      ➜ Tidak ada deteksi\n")

plt.tight_layout()
plt.show()

print("✅ Test inference selesai!")

In [ ]:
# 📦 EXPORT MODEL
# Export ke berbagai format untuk deployment

print("=" * 60)
print("📦 EXPORT MODEL")
print("=" * 60)

print("\n🔄 Exporting model ke berbagai format...")
print("   (Proses ini memakan waktu 2-5 menit)\n")

# 1. ONNX (untuk deployment umum)
print("📤 1. Exporting to ONNX...")
try:
    best_model.export(format='onnx', imgsz=640)
    print("   ✅ ONNX export complete!")
except Exception as e:
    print(f"   ❌ Error: {e}")

# 2. TensorFlow Lite (untuk Raspberry Pi / Mobile)
print("\n📤 2. Exporting to TFLite...")
try:
    best_model.export(format='tflite', imgsz=640)
    print("   ✅ TFLite export complete!")
except Exception as e:
    print(f"   ❌ Error: {e}")

# 3. TorchScript (untuk PyTorch deployment)
print("\n📤 3. Exporting to TorchScript...")
try:
    best_model.export(format='torchscript', imgsz=640)
    print("   ✅ TorchScript export complete!")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n" + "=" * 60)
print("✅ EXPORT SELESAI!")
print("=" * 60)
print("\nFile yang tersedia:")
print(f"   📁 {run_dir}/weights/")
print("      - best.pt (PyTorch)")
print("      - best.onnx (ONNX)")
print("      - best.tflite (TensorFlow Lite)")
print("      - best.torchscript (TorchScript)")

In [ ]:
# 💾 COPY HASIL KE GOOGLE DRIVE
# Agar hasil training tidak hilang saat Colab disconnect

import shutil

print("=" * 60)
print("💾 MENYIMPAN KE GOOGLE DRIVE")
print("=" * 60)

# Path tujuan di Google Drive
drive_save_path = PROJECT_DIR / CONFIG['name']

print(f"\n📁 Copying files...")
print(f"   From: {run_dir}")
print(f"   To: {drive_save_path}")

# Hapus folder lama jika ada
if drive_save_path.exists():
    shutil.rmtree(drive_save_path)

# Copy semua hasil
shutil.copytree(run_dir, drive_save_path)

print(f"\n✅ HASIL TRAINING TERSIMPAN DI GOOGLE DRIVE!")
print(f"📁 Location: {drive_save_path}")

print("\n📦 File yang tersimpan:")
print("   ✅ best.pt - Model terbaik")
print("   ✅ last.pt - Model terakhir")
print("   ✅ best.onnx - Format ONNX")
print("   ✅ best.tflite - Format TFLite")
print("   ✅ results.png - Training plots")
print("   ✅ confusion_matrix.png - Confusion matrix")
print("   ✅ Dan semua file hasil training lainnya")

print("\n💡 File ini akan tetap ada meskipun Colab disconnect!")

In [ ]:
# 📥 DOWNLOAD MODEL
# Download model ke komputer lokal

from google.colab import files

print("=" * 60)
print("📥 DOWNLOAD MODEL KE KOMPUTER")
print("=" * 60)

print("\n💡 Pilih file yang ingin di-download:\n")

download_files = {
    '1': (run_dir / 'weights' / 'best.pt', 'Model PyTorch (WAJIB!)'),
    '2': (run_dir / 'weights' / 'best.onnx', 'Model ONNX'),
    '3': (run_dir / 'weights' / 'best.tflite', 'Model TFLite'),
    '4': (run_dir / 'results.png', 'Training Results'),
    '5': (run_dir / 'confusion_matrix.png', 'Confusion Matrix'),
}

# Download best.pt (WAJIB untuk inference!)
print("⬇️ Downloading best.pt (Model utama)...")
best_pt = run_dir / 'weights' / 'best.pt'
if best_pt.exists():
    files.download(str(best_pt))
    print("✅ best.pt downloaded!")
else:
    print("❌ File tidak ditemukan!")

# Download ONNX (opsional, untuk deployment)
print("\n⬇️ Downloading best.onnx...")
best_onnx = run_dir / 'weights' / 'best.onnx'
if best_onnx.exists():
    files.download(str(best_onnx))
    print("✅ best.onnx downloaded!")

print("\n" + "=" * 60)
print("✅ DOWNLOAD SELESAI!")
print("=" * 60)
print("\n📝 Langkah selanjutnya:")
print("   1. Simpan file best.pt di folder models/")
print("   2. Gunakan untuk inference di laptop")
print("   3. Integrasikan dengan ESP32-CAM")

In [ ]:
# 📊 SUMMARY & NEXT STEPS

print("\n" + "=" * 60)
print("🎉 TRAINING COMPLETE - SUMMARY")
print("=" * 60)

print(f"\n📁 Project: {CONFIG['project']}")
print(f"📝 Name: {CONFIG['name']}")
print(f"🤖 Model: {CONFIG['model']}")
print(f"📊 Dataset: {len(train_images)} train, {len(val_images)} val")
print(f"⚙️ Epochs: {CONFIG['epochs']}")

print(f"\n📈 Final Metrics:")
print(f"   mAP50      : {metrics.box.map50:.4f}")
print(f"   mAP50-95   : {metrics.box.map:.4f}")
print(f"   Precision  : {metrics.box.mp:.4f}")
print(f"   Recall     : {metrics.box.mr:.4f}")

print(f"\n💾 Saved to:")
print(f"   Google Drive: {drive_save_path}")
print(f"   Colab: {run_dir}")

print("\n" + "=" * 60)
print("🚀 NEXT STEPS - UNTUK UAS BESOK:")
print("=" * 60)

print("""
1. ✅ Download best.pt (sudah dilakukan di cell sebelumnya)
2. 📁 Simpan di folder: models/best.pt
3. 🖥️ Setup inference script di laptop
4. 🔧 Setup ESP32 + ESP32-CAM hardware
5. 📱 Setup Blynk untuk monitoring
6. 🧪 Test dengan sampah asli
7. 🎥 Record video demo
8. 📊 Prepare presentasi UAS

💡 TIP PENTING:
   - Jika waktu terbatas, fokus ke inference di laptop dulu
   - ESP32-CAM bisa pakai webcam sebagai backup
   - Blynk bisa di-skip jika tidak sempat
   - Yang penting: MODEL TRAINED + BISA DETEKSI = NILAI BAGUS!

🎓 GOOD LUCK UNTUK UAS BESOK! 🚀
""")

print("=" * 60)